# 필요한 라이브러리 import


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time 

# 유틸리티 함수 정의

In [2]:
# input 데이터와 input 데이터를 한글자씩 뒤로 민 target 데이터를 생성하는 utility 함수를 정의합니다.
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]

  return input_text, target_text

# 설정값 지정

In [3]:
ß# 학습에 필요한 설정값들을 지정합니다.
#data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')  # shakespeare
data_dir = tf.keras.utils.get_file('linux.txt', 'https://raw.githubusercontent.com/solaris33/deep-learning-tensorflow-book-code/master/Ch08-RNN/Char-RNN/data/linux/input.txt')  # linux
batch_size = 64      # Training : 64, Sampling : 1 ##### 64 배치
seq_length = 100     # Training : 100, Sampling : 1 ##### 100글자씩 묶음
embedding_dim = 256  # Embedding 차원
hidden_size = 1024   # 히든 레이어의 노드 개수
num_epochs = 10

# 어휘 집합(Vocabulary set) 설정

In [4]:
# 학습에 사용할 txt 파일을 읽습니다.
text = open(data_dir, 'rb').read().decode(encoding='utf-8')
# 학습데이터에 포함된 모든 character들을 나타내는 변수인 vocab과
# vocab에 id를 부여해 dict 형태로 만든 char2idx를 선언합니다.
vocab = sorted(set(text))  # 유니크한 character 개수
vocab_size = len(vocab)
print('{} unique characters'.format(vocab_size))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

99 unique characters


In [5]:
vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '©',
 'å']

In [6]:
char2idx

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '^': 64,
 '_': 65,
 '`': 66,
 'a': 67,
 'b': 68,
 'c': 69,
 'd': 70,
 'e': 71,
 'f': 72,
 'g': 73,
 'h': 74,
 'i': 75,
 'j': 76,
 'k': 77,
 'l': 78,
 'm': 79,
 'n': 80,
 'o': 81,
 'p': 82,
 'q': 83,
 'r': 84,
 's': 85,
 't': 86,
 'u': 87,
 'v': 88,
 'w': 89,
 'x': 90,
 'y': 91,
 'z': 92,
 '{': 93,
 '|': 94,
 '}': 95,
 '~': 96,
 '©': 97,
 'å': 98}

In [7]:
idx2char

array(['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
       '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
       'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^',
       '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
       'y', 'z', '{', '|', '}', '~', '©', 'å'], dtype='<U1')

# Dataset 설정

In [8]:
# 학습 데이터를 character에서 integer로 변환합니다.
text_as_int = np.array([char2idx[c] for c in text])

# split_input_target 함수를 이용해서 input 데이터와 input 데이터를 한글자씩 뒤로 민 target 데이터를 생성합니다.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

# RNN 모델 설정

In [9]:
# tf.keras.Model을 이용해서 RNN 모델을 정의합니다.
class RNN(tf.keras.Model):
 def __init__(self, batch_size):
   super(RNN, self).__init__()
   self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                                    batch_input_shape=[batch_size, None])
   self.hidden_layer_1 = tf.keras.layers.LSTM(hidden_size,
                                             return_sequences=True,
                                             stateful=True,
                                             recurrent_initializer='glorot_uniform')
   self.output_layer = tf.keras.layers.Dense(vocab_size)

 def call(self, x):
   embedded_input = self.embedding_layer(x)
   features = self.hidden_layer_1(embedded_input)
   logits = self.output_layer(features)

   return logits
 

# vocab size 99 dimension (one-hot encoding 된 벡터)를 256 임베딩으로 맞춰줌
# 일반적으로는 vocab size가 임베딩보다 훨씬 큼
# 이 경우는 char 단위로 하다보니 embedding dim 이 더 큼
# 다음 one-hot encoding된 char 에 대한 softmax regression 값을 call 함수에서 output으로 내줌



# Loss Function 정의

In [10]:
# sparse cross-entropy 손실 함수를 정의합니다.
def sparse_cross_entropy_loss(labels, logits):
  return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True))

# 옵티마이저 및 학습 설정

In [11]:
# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.keras.optimizers.Adam()

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, input, target):
  with tf.GradientTape() as tape:
    logits = model(input)
    loss = sparse_cross_entropy_loss(target, logits)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

# 샘플링 함수 설정

In [12]:
def generate_text(model, start_string):
  num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다.

  # start_sting을 integer 형태로 변환합니다.
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 샘플링 결과로 생성된 string을 저장할 배열을 초기화합니다.
  text_generated = []

  # 낮은 temperature 값은 더욱 정확한 텍스트를 생성합니다.
  # 높은 temperature 값은 더욱 다양한 텍스트를 생성합니다.
  ##### Temp 가 높을수록 편차가 높아짐 (argmax와 비슷하게 되는 경향), 정확성 오르고 다양상은 떨어짐
  temperature = 1.0

  # 여기서 batch size = 1 입니다.
  model.reset_states()
  for i in range(num_sampling):
    predictions = model(input_eval)
    # 불필요한 batch dimension을 삭제합니다.
    predictions = tf.squeeze(predictions, 0)

    # 모델의 예측결과에 기반해서 랜덤 샘플링을 하기위해 categorical distribution을 사용합니다.
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 예측된 character를 다음 input으로 사용합니다.
    input_eval = tf.expand_dims([predicted_id], 0)
    # 샘플링 결과를 text_generated 배열에 추가합니다.
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

# 트레이닝 시작

In [13]:
# Recurrent Neural Networks(RNN) 모델을 선언합니다.
RNN_model = RNN(batch_size=batch_size)

# 데이터 구조 파악을 위해서 예제로 임의의 하나의 배치 데이터 에측하고, 예측결과를 출력합니다.
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = RNN_model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# 모델 정보를 출력합니다.
RNN_model.summary()

# checkpoint 데이터를 저장할 경로를 지정합니다.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

for epoch in range(num_epochs):
  start = time.time()

  # 매 반복마다 hidden state를 초기화합니다. (최초의 hidden 값은 None입니다.)
  hidden = RNN_model.reset_states()

  for (batch_n, (input, target)) in enumerate(dataset):
    loss = train_step(RNN_model, input, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
  if (epoch + 1) % 5 == 0:
    RNN_model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

RNN_model.save_weights(checkpoint_prefix.format(epoch=epoch))
print("트레이닝이 끝났습니다!")

ValueError: Unrecognized keyword arguments passed to Embedding: {'batch_input_shape': [64, None]}

# 트레이닝이 끝난 모델을 이용한 샘플링

In [ ]:
sampling_RNN_model = RNN(batch_size=1)
sampling_RNN_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
sampling_RNN_model.build(tf.TensorShape([1, None]))
sampling_RNN_model.summary()

# 샘플링을 시작합니다.
print("샘플링을 시작합니다!")
print(generate_text(sampling_RNN_model, start_string=u' '))

Model: "rnn_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  25344     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_3 (Dense)              multiple                  101475    
Total params: 5,373,795
Trainable params: 5,373,795
Non-trainable params: 0
_________________________________________________________________
샘플링을 시작합니다!
 ss           printf(" .. MAX_LOCK_USASHIC_PRINT_TASK_TIMEKEEntry))
		if (diag) {
		reset_free_percpu(skb);
		again_ops->proc_cap_buffer = bytesn;
	}

	/*
	 * Determine if CONFIG_SCHEDSTATS */

	/* Keep start */
	dr->flags &= ~SPONT_PAREN; i++) {
		kdb_printf("kdb enabled" by this code trigger that may have been actually. If all
 * threads can possibly get_kprojid to